In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Loading Data

In [ ]:
import pandas as pd

df_train = pd.read_csv('/content/drive/My Drive/Marketing Analytics/train.csv', index_col='user_id', 
                       na_values=['Never checkout', 'Never open', 'Never login'])

df_users = pd.read_csv('/content/drive/My Drive/Marketing Analytics/users.csv', index_col='user_id')

# Merge User and Email specs into one
df_train = pd.merge(df_train, df_users, how='left', left_index=True, right_index=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder_X = LabelEncoder()
df_train['domain']=labelencoder_X.fit_transform(df_train['domain'])

In [ ]:
df_train.drop(columns=['grass_date','row_id'], inplace=True)
df_train.fillna(0, inplace = True)
df_train.astype('int32')
df_train.shape

(73539, 20)

In [ ]:
df_train

,country_code,subject_line_length,last_open_day,last_login_day,last_checkout_day,open_count_last_10_days,open_count_last_30_days,open_count_last_60_days,login_count_last_10_days,login_count_last_30_days,login_count_last_60_days,checkout_count_last_10_days,checkout_count_last_30_days,checkout_count_last_60_days,open_flag,attr_1,attr_2,attr_3,age,domain
user_id,,,,,,,,,,,,,,,,,,,,
2,6,35,85.0,9.0,0.0,0,0,0,17,51,88,0,0,0,0,0.0,1.0,0.0,0.0,10
4,1,64,59.0,1.0,3.0,0,0,1,27,79,156,4,8,9,0,1.0,1.0,2.0,33.0,1
7,6,50,40.0,9.0,253.0,0,0,3,2,2,6,0,0,0,0,1.0,1.0,1.0,36.0,1
8,5,35,2.0,1.0,2.0,1,3,3,31,106,203,2,4,13,0,1.0,1.0,1.0,43.0,8
9,6,40,29.0,3.0,6.0,0,1,2,4,4,22,1,1,5,0,0.0,1.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127918,1,49,2.0,21.0,0.0,1,4,4,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1
127920,2,41,22.0,3.0,27.0,0,2,4,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1
127923,3,38,20.0,5.0,0.0,0,3,7,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1


Convert data to features and target

In [ ]:
target = ['open_flag'] 
features = list(set(list(df_train.columns))-set(target))

X = df_train[features].values
y = df_train[target].values

Split into Train-Validation

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=40) #10% of validation
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((66185, 19), (7354, 19), (66185, 1), (7354, 1))

Define Model

In [ ]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

#model = RandomForestClassifier(n_estimators=1000)
model = AdaBoostClassifier(n_estimators=1000)

Training

In [ ]:
model = model.fit(X_train, y_train.ravel())

Evaluation

In [ ]:
y_pred = model.predict(X_val)

In [ ]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# Evaluate model
from sklearn.metrics import accuracy_score, matthews_corrcoef, classification_report

accuracy_score = accuracy_score(y_val, y_pred)
matthews_score = matthews_corrcoef(y_val, y_pred)
print(f'Accuracy Score = {accuracy_score}')
print(f'MCC Score = {matthews_score}')
print(classification_report(y_val, y_pred))

Accuracy Score = 0.8821049768833288
MCC Score = 0.49201122492139693
              precision    recall  f1-score   support

           0       0.90      0.96      0.93      6209
           1       0.69      0.45      0.54      1145

    accuracy                           0.88      7354
   macro avg       0.80      0.71      0.74      7354
weighted avg       0.87      0.88      0.87      7354



Load the Test Set

In [ ]:
import pandas as pd

df_test = pd.read_csv("/content/drive/My Drive/Marketing Analytics/test.csv", index_col='user_id', 
                       na_values=['Never checkout', 'Never open', 'Never login'])
#merge with users
df_test = pd.merge(df_test, df_users, how='left', left_index=True, right_index=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder_X = LabelEncoder()
df_test['domain']=labelencoder_X.fit_transform(df_test['domain'])

In [ ]:
df_test.drop(columns=['grass_date','row_id'], inplace=True)
df_test.fillna(0, inplace = True)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55970 entries, 0 to 127895
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   country_code                 55970 non-null  int64  
 1   subject_line_length          55970 non-null  int64  
 2   last_open_day                55970 non-null  float64
 3   last_login_day               55970 non-null  float64
 4   last_checkout_day            55970 non-null  float64
 5   open_count_last_10_days      55970 non-null  int64  
 6   open_count_last_30_days      55970 non-null  int64  
 7   open_count_last_60_days      55970 non-null  int64  
 8   login_count_last_10_days     55970 non-null  int64  
 9   login_count_last_30_days     55970 non-null  int64  
 10  login_count_last_60_days     55970 non-null  int64  
 11  checkout_count_last_10_days  55970 non-null  int64  
 12  checkout_count_last_30_days  55970 non-null  int64  
 13  checkout_count_

In [ ]:
test_dataset = df_test.values

Prediction

In [ ]:
results = model.predict(test_dataset)

In [ ]:
row_id = []
id = 0
for i in results:
  row_id.append(id)
  id += 1

df_result = pd.DataFrame({'row_id': row_id,
                           'open_flag': results})
df_result

,row_id,open_flag
0,0,0
1,1,0
2,2,1
3,3,0
4,4,0
...,...,...
55965,55965,0
55966,55966,0
55967,55967,0
55968,55968,0


In [ ]:
df_result.to_csv('submission.csv', index=False)